# AOC 2025 Day 11

In [1]:
@file:DependsOn("com.toldoven.aoc:aoc-kotlin-notebook:1.1.2")

In [2]:
import com.toldoven.aoc.notebook.AocClient

val aocClient = AocClient.fromEnv().interactiveDay(2025, 11)
val input = aocClient.input()
aocClient.viewPartOne()

--- Day 11: Reactor --- 
 You hear some loud beeping coming from a hatch in the floor of the factory, so you decide to check it out. Inside, you find several large electrical conduits and a ladder. 
 Climbing down the ladder, you discover the source of the beeping : a large, toroidal reactor which powers the factory above. Some Elves here are hurriedly running between the reactor and a nearby server rack, apparently trying to fix something. 
 One of the Elves notices you and rushes over. "It's a good thing you're here! We just installed a new server rack , but we aren't having any luck getting the reactor to communicate with it!" You glance around the room and see a tangle of cables and devices running from the server rack to the reactor. She rushes off, returning a moment later with a list of the devices and their outputs (your puzzle input). 
 For example: 
 aaa: you hhh
you: bbb ccc
bbb: ddd eee
ccc: ddd eee fff
ddd: ggg
eee: out
fff: out
ggg: out
hhh: ccc fff iii
iii: out
 
 Each line gives the name of a device followed by a list of the devices to which its outputs are attached. So, bbb: ddd eee means that device bbb has two outputs, one leading to device ddd and the other leading to device eee . 
 The Elves are pretty sure that the issue isn't due to any specific device, but rather that the issue is triggered by data following some specific path through the devices. Data only ever flows from a device through its outputs; it can't flow backwards. 
 After dividing up the work, the Elves would like you to focus on the devices starting with the one next to you (an Elf hastily attaches a label which just says you ) and ending with the main output to the reactor (which is the device with the label out ). 
 To help the Elves figure out which path is causing the issue, they need you to find every path from you to out . 
 In this example, these are all of the paths from you to out : 
 
 Data could take the connection from you to bbb , then from bbb to ddd , then from ddd to ggg , then from ggg to out . 
 Data could take the connection to bbb , then to eee , then to out . 
 Data could go to ccc , then ddd , then ggg , then out . 
 Data could go to ccc , then eee , then out . 
 Data could go to ccc , then fff , then out . 
 
 In total, there are 5 different paths leading from you to out . 
 How many different paths lead from you to out ?

In [4]:
import org.apache.xpath.functions.FuncCurrent

typealias Graph = Map<String, List<String>>

val lines = input.lines()
val graph = lines
    .associate { line ->
        val (node, connections) = line.split(": ", limit = 2)
        val neighbors = connections.split(" ")
        node to neighbors
    }

// start at graph["you"] and count paths that lead to graph["out"]
val cache = mutableMapOf<String, Long>()

fun countPaths(current: String): Long {
    if (current == "out") return 1L
    // return the cached value if already computed
    cache[current]?.let { return it }

    val neighbors = graph[current] ?: emptyList()

    val totalPaths = neighbors.sumOf { neighbor ->
        countPaths(neighbor)
    }

    cache[current] = totalPaths
    return totalPaths
}

val answerPart1 = countPaths("you")
aocClient.submitPartOne(answerPart1)

Your answer: 696. 
 That's the right answer! You are one gold star closer to decorating the North Pole. [Continue to Part Two]

In [5]:
aocClient.viewPartTwo()

--- Part Two --- 
 Thanks in part to your analysis, the Elves have figured out a little bit about the issue. They now know that the problematic data path passes through both dac (a digital-to-analog converter ) and fft (a device which performs a fast Fourier transform ). 
 They're still not sure which specific path is the problem, and so they now need you to find every path from svr (the server rack) to out . However, the paths you find must all also visit both dac and fft (in any order). 
 For example: 
 svr: aaa bbb
aaa: fft
fft: ccc
bbb: tty
tty: ccc
ccc: ddd eee
ddd: hub
hub: fff
eee: dac
dac: fff
fff: ggg hhh
ggg: out
hhh: out
 
 This new list of devices contains many paths from svr to out : 
 svr,aaa, fft ,ccc,ddd,hub,fff,ggg,out
svr,aaa, fft ,ccc,ddd,hub,fff,hhh,out
svr,aaa, fft ,ccc,eee, dac ,fff,ggg,out
svr,aaa, fft ,ccc,eee, dac ,fff,hhh,out
svr,bbb,tty,ccc,ddd,hub,fff,ggg,out
svr,bbb,tty,ccc,ddd,hub,fff,hhh,out
svr,bbb,tty,ccc,eee, dac ,fff,ggg,out
svr,bbb,tty,ccc,eee, dac ,fff,hhh,out
 
 However, only 2 paths from svr to out visit both dac and fft . 
 Find all of the paths that lead from svr to out . How many of those paths visit both dac and fft ?

In [7]:
val memo = mutableMapOf<Pair<String, String>, Long>()

fun count(from: String, to: String): Long {
    if (from == to) return 1
    val state = from to to
    memo[state]?.let { return it }

    val neighbors = graph[from] ?: emptyList()
    val total = neighbors.sumOf { count(it, to) }

    memo[state] = total
    return total
}

// Scenario 1: svr -> dac -> fft -> out
val sequence1 = count("svr", "dac") * count("dac", "fft") * count("fft", "out")
// Scenario 2: svr -> fft -> dac -> out
val sequence2 = count("svr", "fft") * count("fft", "dac") * count("dac", "out")

val answerPart2 = sequence1 + sequence2
aocClient.submitPartTwo(answerPart2)

Your answer: 473741288064360. 
 That's the right answer! You are one gold star closer to decorating the North Pole. 
 You have completed Day 11! You can [Share on Bluesky Twitter Mastodon ] this victory or [Return to Your Advent Calendar] .